** Commence absured amount of imports!** 

In [1]:
%matplotlib notebook

# standard imports 
from __future__ import division 
from __future__ import print_function 
import matplotlib.pyplot as plt 
import numpy as np 
import os 
from glob import glob 

In [2]:
# imports for playing with fits files 
from sunpy.net import Fido, attrs
from datetime import datetime, timedelta
import astropy.units as u 
import sunpy.map as smap 
from astropy.coordinates import SkyCoord 
from scipy.interpolate import interp1d
from astropy.convolution import convolve, Box1DKernel

In [3]:
from AIA_tools import simulation, analysis

** Let's see if the sumbap function works ** 

In [ ]:
subMap = analysis.makeSubmap(data_maps[30], 900, -500, 1200, -200 )

In [ ]:
fig = plt.figure() 
ax = plt.subplot(projection = subMap)
subMap.plot(axes=ax) 
ax.plot_coord(radial_slits[30], color='c', linewidth=0.5)

plt.show() 

** Now let's get the slit intensities ** 

In [ ]:
raw_slit_intensities = []
distances = [] 

for i in range(len(radial_slits)): 
    intensity_along_slit, distance = analysis.getRadialSlitIntensity(radial_slits[i], data_maps[i])
    raw_slit_intensities.append(intensity_along_slit)
    distances.append(distance) 
    
print(len(distances), len(raw_slit_intensities))

In [ ]:
smoothed_intensities = [] 

for intens in raw_slit_intensities: 
        smoothed_intensities.append(analysis.smoothAndNormalizeSlit(intens)) 

In [ ]:
plt.figure() 
plt.plot(distances[12], smoothed_intensities[12])
plt.xlabel('Distance [$R_\odot$]')
plt.ylabel('Normalized Intensity')
plt.title('Intensity along radial slit 12')
plt.show() 

** Now let's stack all of the slit images into a 2-d figure ** 

In [ ]:
fig1 = plt.figure(figsize=(20,10)) 
ax1 = plt.subplot(121, projection = subMap)
subMap.plot(cmap=plt.cm.Greys)
ax2 = plt.subplot(122) 
ax2.plot(distances[12], smoothed_intensities[12])
ax2.set_xlabel('x label') 
ax2.set_title('title')

plt.show() 

In [ ]:
times = analysis.getTimes(data_maps)

In [ ]:
import matplotlib.dates as mdates

shape = np.shape(distances) 
distances = np.asarray(distances) 
assert(np.shape(distances) == np.shape(smoothed_intensities))
ts = [[times[i] for i in range(shape[0])] for j in range(shape[1])]
ts = np.asarray(ts)

print(np.shape(ts), shape, np.shape(smoothed_intensities))

fig, ax = plt.subplots() 
ax.pcolormesh(ts, np.transpose(distances), np.transpose(smoothed_intensities))
fig.autofmt_xdate()
myFmt = mdates.DateFormatter('%H:%M:%S')
ax.xaxis.set_major_formatter(myFmt)
plt.show() 

** try to replicate with the function ** 

In [ ]:
import matplotlib 
fig, ax = plt.subplots()  
analysis.plotRadialStack(times, distances, smoothed_intensities, fig, ax)
ax.set_xlabel("observation time", fontsize=18)
ax.set_ylabel("Distance $[R_\odot]$", fontsize=18)
ax.set_title("Radial Slit Intensity - 193 $\AA$", fontsize=18) 
matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=8) 






plt.show() 

del data_maps

In [ ]:
fits_193 = glob("/data/khnum/REU2018/jwaczak/fitsFiles/193/*.fits")
fits_171 = glob("/data/khnum/REU2018/jwaczak/fitsFiles/171/*.fits")
fits_211 = glob("/data/khnum/REU2018/jwaczak/fitsFiles/211/*.fits")
fits_304 = glob("/data/khnum/REU2018/jwaczak/fitsFiles/304/*.fits")
fits_335 = glob("/data/khnum/REU2018/jwaczak/fitsFiles/335/*.fits")

In [ ]:
x_i = 990*u.arcsec 
y_i = -450*u.arcsec 
x_f = 1200*u.arcsec 
y_f = -265*u.arcsec 

radial_slits_171 = [] 
radial_slits_193 = []
radial_slits_211 = []
radial_slits_304 = [] 
radial_slits_335 = [] 

assert(len(i) == len(j) for i,j in [fits_193, fits_171, fits_211, fits_304, fits_335])

for i in range(len(fits_171)): 
    if i%5==0: print(i) 
    map_171 = smap.Map(fits_171[i])
    map_193 = smap.Map(fits_193[i])
    map_211 = smap.Map(fits_211[i])
    map_304 = smap.Map(fits_304[i])
    map_335 = smap.Map(fits_335[i])
    
    radial_slits_171.append(analysis.getRadialSlit(x_i, x_f, y_i, y_f, 200, map_171))
    radial_slits_193.append(analysis.getRadialSlit(x_i, x_f, y_i, y_f, 200, map_193))
    radial_slits_211.append(analysis.getRadialSlit(x_i, x_f, y_i, y_f, 200, map_211))
    radial_slits_304.append(analysis.getRadialSlit(x_i, x_f, y_i, y_f, 200, map_304))
    radial_slits_335.append(analysis.getRadialSlit(x_i, x_f, y_i, y_f, 200, map_335))
    
    del map_171
    del map_193
    del map_211
    del map_304
    del map_335

In [ ]:
print(radial_slits_171)

In [ ]:
slit_intensities = {'171':[], '193':[], '211':[], '304':[], '335':[]}
distances = {'171':[], '193':[], '211':[], '304':[], '335':[]}

print(len(radial_slits_171))
for i in range(len(radial_slits_171)): 
        if i%5==0: print(i) 
        i171, d171 = analysis.getRadialSlitIntensity(radial_slits_171[i], smap.Map(fits_171[i]))
        i193, d193 = analysis.getRadialSlitIntensity(radial_slits_193[i], smap.Map(fits_193[i]))
        i211, d211 = analysis.getRadialSlitIntensity(radial_slits_211[i], smap.Map(fits_211[i]))
        i304, d304 = analysis.getRadialSlitIntensity(radial_slits_304[i], smap.Map(fits_304[i]))
        i335, d335 = analysis.getRadialSlitIntensity(radial_slits_335[i], smap.Map(fits_335[i]))
        
        slit_intensities['171'].append(analysis.smoothAndNormalizeSlit(i171))
        distances['171'].append(d171)
        
        slit_intensities['193'].append(analysis.smoothAndNormalizeSlit(i193))
        distances['193'].append(d193)
        
        slit_intensities['211'].append(analysis.smoothAndNormalizeSlit(i211))
        distances['211'].append(d211)
        
        slit_intensities['304'].append(analysis.smoothAndNormalizeSlit(i304))
        distances['304'].append(d304)
        
        slit_intensities['335'].append(analysis.smoothAndNormalizeSlit(i335))
        distances['335'].append(d335)
        


In [ ]:
times_171 = analysis.getTimes(smap.Map(fits_171))
times_193 = analysis.getTimes(smap.Map(fits_193))
times_211 = analysis.getTimes(smap.Map(fits_211))
times_304 = analysis.getTimes(smap.Map(fits_304))
times_335 = analysis.getTimes(smap.Map(fits_335))

In [ ]:
import matplotlib
with matplotlib.pyplot.style.context(("dark_background")):
    fig, ax = plt.subplots(nrows=1, ncols=5, figsize=(20,8)) 

    analysis.plotRadialStack(times_171, distances['171'], slit_intensities['171'], fig, ax[0])
    ax[0].set_xlabel("time", fontsize=18)
    ax[0].set_ylabel("Distance $[R_\odot]$", fontsize=18)
    ax[0].set_title("171 $\AA$", fontsize=18)

    analysis.plotRadialStack(times_193, distances['193'], slit_intensities['193'], fig, ax[1])
    ax[1].set_xlabel("time", fontsize=18)
    ax[1].set_ylabel("Distance $[R_\odot]$", fontsize=18)
    ax[1].set_title("193 $\AA$", fontsize=18)

    analysis.plotRadialStack(times_211, distances['211'], slit_intensities['211'], fig, ax[2])
    ax[2].set_xlabel("time", fontsize=18)
    ax[2].set_ylabel("Distance $[R_\odot]$", fontsize=18)
    ax[2].set_title("211 $\AA$", fontsize=18)

    analysis.plotRadialStack(times_304, distances['304'], slit_intensities['304'], fig, ax[3])
    ax[3].set_xlabel("time", fontsize=18)
    ax[3].set_ylabel("Distance $[R_\odot]$", fontsize=18)
    ax[3].set_title("304 $\AA$", fontsize=18)

    analysis.plotRadialStack(times_335, distances['335'], slit_intensities['335'], fig, ax[4])
    ax[4].set_xlabel("time", fontsize=18)
    ax[4].set_ylabel("Distance $[R_\odot]$", fontsize=18)
    ax[4].set_title("335 $\AA$", fontsize=18)
  
    plt.tight_layout()

    ax[0].tick_params(axis='both', which='major', labelsize=10)
    ax[1].tick_params(axis='both', which='major', labelsize=10)
    ax[2].tick_params(axis='both', which='major', labelsize=10)
    ax[3].tick_params(axis='both', which='major', labelsize=10)
    ax[4].tick_params(axis='both', which='major', labelsize=10)

    fig.suptitle('Intensity along radial slit', fontsize=18)
    plt.subplots_adjust(top=0.85)
    
    plt.savefig('/data/khnum/REU2018/jwaczak/data/intensityAlongSlit.png', transparent=True)
    plt.show() 



In [6]:
pathToBackground = '/home/john/gitRepos/REU/jwaczak/data/background.txt'
N_w = 30
k=1
background_new = simulation.fitBackgroundEmissivity(pathToBackground, simulation.R_w, simulation.R_sun, N_w, k=k, R_max = 2, tol=0.01)

Background: [42.10212355 44.31490303  9.5362187   0.99588045  0.32414652]
(-732220831369735.0, 0.421021235508)
(-732220831369732.8, 0.443149030324)
(-732220831369767.6, 0.09536218704919999)
(-732220831369776.1, 0.00995880448586)
(-732220831369776.8, 0.00324146522877)


In [ ]:
print(background_new)

In [ ]:
40.0/(2*(10**8)**2*1.5*696000*(1/3600)**2*(np.pi/180)**2 * 1.5**2*10**5)

In [ ]:
radii = np.loadtxt('/home/john/gitRepos/REU/jwaczak/data/initialRadii.txt', delimiter=',')

In [ ]:
print(radii)


In [10]:
A = np.array([1, 2, 3, 4, 5, 6])
B = 1.2
print(analysis.getNearestValue(A, B))

0


In [12]:
print("{0:.2f}, {1:.2f}".format(3.14159, 2.71717171))

3.14, 2.72
